In [14]:
import pandas as pd
import numpy as np
import json

In [15]:
offering_df_dic = dict()
for colum_l in ['hotel_class', 'region_id', 'url', 'phone', 'details', 'address', 'type', 'id', 'name']:
    offering_df_dic[colum_l] = []

f = open("offering.txt", "r")
for line in f.readlines():
    temp_dic = json.loads(line)
    for i in ['hotel_class', 'region_id', 'url', 'phone', 'details', 'address', 'type', 'id', 'name']:
        if 'hotel_class' not in list(temp_dic.keys()) and i == 'hotel_class':
            value = np.nan
        else:
            value = temp_dic[i]
        offering_df_dic[i].append(value)
f.close()

In [16]:
offering_df = pd.DataFrame.from_dict(offering_df_dic)

In [17]:
for i in range(len(offering_df)):
    temp_dic = dict(offering_df.iloc[i]['address'])
    for key in ['region', 'street-address', 'postal-code', 'locality']:
        try:
            offering_df.loc[i,key] = temp_dic[key]
        except KeyError:
            offering_df.loc[i,key] = np.nan

In [18]:
f = open("review.txt", "r")
count = 0
for line in f.readlines():
    temp_dic = json.loads(line)
    temp_dic = temp_dic['ratings']
    count +=1 

In [19]:
rating_dict = {}
for i in ['service', 'cleanliness', 'overall', 'value', 'location', 'sleep_quality', 'rooms']:
    rating_dict[i] = []

In [20]:
f = open("review.txt", "r")
for line in f.readlines():
    temp_dic = json.loads(line)
    temp_dic = temp_dic['ratings']
    for rating_key in ['service', 'cleanliness', 'overall', 'value', 'location', 'sleep_quality', 'rooms']:
        try:
            value = temp_dic[rating_key]
            rating_dict[rating_key].append(value)
        except KeyError:
            rating_dict[rating_key].append(np.nan)
f.close()

In [21]:
author_dict = {}
for i in ['username', 'num_cities', 'num_helpful_votes', 'num_reviews', 'num_type_reviews', 'id']:
    author_dict[i] = []

In [22]:
f = open("review.txt", "r")
for line in f.readlines():
    temp_dic = json.loads(line)
    temp_dic = temp_dic['author']
    for author_key in ['username', 'num_cities', 'num_helpful_votes',
                       'num_reviews', 'num_type_reviews', 'id']:
        try:
            value = temp_dic[author_key]
            author_dict[author_key].append(value)
        except KeyError:
            author_dict[author_key].append(np.nan)
f.close()

In [23]:
review_dict = {}
for i in ['title', 'text', 'date_stayed', 'offering_id',
          'num_helpful_votes', 'date', 'id', 'via_mobile']:
    review_dict[i] = []

In [24]:
f = open("review.txt", "r")
for line in f.readlines():
    temp_dic = json.loads(line)
    for review_key in ['title', 'text', 'date_stayed', 'offering_id',
          'num_helpful_votes', 'date', 'id', 'via_mobile']:
        try:
            value = temp_dic[review_key]
            review_dict[review_key].append(value)
        except KeyError:
            review_dict[review_key].append(np.nan)
f.close()

In [25]:
# change the name of dictionary 
author_dict['author_num_helpful_votes'] = author_dict.pop('num_helpful_votes')
author_dict['author_id'] = author_dict.pop('id')

In [26]:
# merge the three dictionaries
def Merge(dict1, dict2):
    return(dict2.update(dict1))

In [27]:
Merge(author_dict, review_dict)

In [28]:
Merge(rating_dict, review_dict)

In [29]:
review_df = pd.DataFrame.from_dict(review_dict)

In [30]:
df = review_df.merge(offering_df,left_on='offering_id', right_on='id')

In [31]:
df.isna().sum()

title                            0
text                             0
date_stayed                  67588
offering_id                      0
num_helpful_votes                0
date                             0
id_x                             0
via_mobile                       0
username                         0
num_cities                  264593
num_reviews                  77123
num_type_reviews            362951
author_num_helpful_votes    224581
author_id                        0
service                     117643
cleanliness                 118726
overall                          0
value                       124866
location                    213657
sleep_quality               377658
rooms                       173157
hotel_class                  34937
region_id                        0
url                              0
phone                            0
details                     878561
address                          0
type                             0
id_y                

In [32]:
# Drop details because no details is provided for all reviews
# Consider nums to be 0 if original values are nan
df = df.drop('details', axis=1)
df['num_cities'] = df['num_cities'].replace(np.nan, 0)
df['num_reviews'] = df['num_reviews'].replace(np.nan, 0)
df['num_type_reviews'] = df['num_type_reviews'].replace(np.nan, 0)
df['author_num_helpful_votes'] = df['author_num_helpful_votes'].replace(np.nan, 0)
df = df.drop('date_stayed', axis=1)

In [33]:
# fill 'date_stayed' using the value from date, usually those two are the same
# I think we should be fine to drop 'date_stayed' and use 'date' will be fine

# date_index = df[df['date_stayed'].isnull()].index

# for i in date_index:
#   date_strs = df['date'][0].split(", ")
#   extracted_date = date_strs[0][:-2] + date_strs[1]
#   df.loc[i, 'date_stayed'] = extracted_date

In [34]:
service_index = df[df['service'].isnull()].index
service_means = []

for i in df['overall'].unique():
    service_means.append(df[df['overall'] == i]['service'].mean())

service_means.reverse()

for i in service_index:
    cur_overall = int(df.loc[i]['overall'])
    df.loc[i, 'service'] = service_means[cur_overall]

In [35]:
cleanliness_index = df[df['cleanliness'].isnull()].index
cleanliness_means = []

for i in df['overall'].unique():
    cleanliness_means.append(df[df['overall'] == i]['cleanliness'].mean())

cleanliness_means.reverse()

for i in cleanliness_index:
    cur_overall = int(df.loc[i]['overall'])
    df.loc[i, 'cleanliness'] = cleanliness_means[cur_overall]

In [36]:
df.isnull().sum()

title                            0
text                             0
offering_id                      0
num_helpful_votes                0
date                             0
id_x                             0
via_mobile                       0
username                         0
num_cities                       0
num_reviews                      0
num_type_reviews                 0
author_num_helpful_votes         0
author_id                        0
service                          0
cleanliness                      0
overall                          0
value                       124866
location                    213657
sleep_quality               377658
rooms                       173157
hotel_class                  34937
region_id                        0
url                              0
phone                            0
address                          0
type                             0
id_y                             0
name                             0
region              

In [37]:
value_index = df[df['value'].isnull()].index
value_means = []

for i in df['overall'].unique():
    value_means.append(df[df['overall'] == i]['value'].mean())

value_means.reverse()

for i in value_index:
    cur_overall = int(df.loc[i]['overall'])
    df.loc[i, 'value'] = value_means[cur_overall]

In [38]:
location_index = df[df['location'].isnull()].index
location_means = []

for i in df['overall'].unique():
    location_means.append(df[df['overall'] == i]['location'].mean())

location_means.reverse()

for i in location_index:
    cur_overall = int(df.loc[i]['overall'])
    df.loc[i, 'location'] = location_means[cur_overall]

In [39]:
sleep_index = df[df['sleep_quality'].isnull()].index
sleep_means = []

for i in df['overall'].unique():
    sleep_means.append(df[df['overall'] == i]['sleep_quality'].mean())

sleep_means.reverse()

for i in sleep_index:
    cur_overall = int(df.loc[i]['overall'])
    df.loc[i, 'sleep_quality'] = sleep_means[cur_overall]

In [40]:
rooms_index = df[df['rooms'].isnull()].index
rooms_means = []

for i in df['overall'].unique():
    rooms_means.append(df[df['overall'] == i]['rooms'].mean())

rooms_means.reverse()

for i in rooms_index:
    cur_overall = int(df.loc[i]['overall'])
    df.loc[i, 'rooms'] = rooms_means[cur_overall]

In [41]:
df.isnull().sum()

title                           0
text                            0
offering_id                     0
num_helpful_votes               0
date                            0
id_x                            0
via_mobile                      0
username                        0
num_cities                      0
num_reviews                     0
num_type_reviews                0
author_num_helpful_votes        0
author_id                       0
service                         0
cleanliness                     0
overall                         0
value                           0
location                       17
sleep_quality                  17
rooms                           0
hotel_class                 34937
region_id                       0
url                             0
phone                           0
address                         0
type                            0
id_y                            0
name                            0
region                          0
street-address

In [43]:
# the username, author_id and phone have empty string, need to be treated as NA 
df.iloc[1001]

title                              “An Excellent Value with a Great Location”
text                        My spouse and I rely on Trip Advisor for guida...
offering_id                                                             93338
num_helpful_votes                                                          18
date                                                            July 16, 2006
id_x                                                                  5472451
via_mobile                                                              False
username                                                                     
num_cities                                                                  0
num_reviews                                                                 0
num_type_reviews                                                            0
author_num_helpful_votes                                                    0
author_id                                                       

In [44]:
# we dont need the phone variable, we just drop it 
df = df.drop('phone',axis=1)

In [48]:
# since the username and author_id is unique, and basically the same thing, we just fill using the index 
df.loc[df.username == "",'username'] = np.nan
df.loc[df.author_id == "",'author_id'] = np.nan

In [51]:
df.isna().sum()

title                           0
text                            0
offering_id                     0
num_helpful_votes               0
date                            0
id_x                            0
via_mobile                      0
username                    77067
num_cities                      0
num_reviews                     0
num_type_reviews                0
author_num_helpful_votes        0
author_id                   77066
service                         0
cleanliness                     0
overall                         0
value                           0
location                       17
sleep_quality                  17
rooms                           0
hotel_class                 34937
region_id                       0
url                             0
address                         0
type                            0
id_y                            0
name                            0
region                          0
street-address                  9
postal-code   

In [55]:
df.loc[df['username'].isnull(), 'username'] = df.loc[df['username'].isnull()].index
df.loc[df['author_id'].isnull(), 'author_id'] = df.loc[df['author_id'].isnull()].index

In [56]:
df.isna().sum()

title                           0
text                            0
offering_id                     0
num_helpful_votes               0
date                            0
id_x                            0
via_mobile                      0
username                        0
num_cities                      0
num_reviews                     0
num_type_reviews                0
author_num_helpful_votes        0
author_id                       0
service                         0
cleanliness                     0
overall                         0
value                           0
location                       17
sleep_quality                  17
rooms                           0
hotel_class                 34937
region_id                       0
url                             0
address                         0
type                            0
id_y                            0
name                            0
region                          0
street-address                  9
postal-code   

In [57]:
# hotel class with na, just ignore those 
df = df[df['hotel_class'].notna()]

In [58]:
df.isna().sum()

title                          0
text                           0
offering_id                    0
num_helpful_votes              0
date                           0
id_x                           0
via_mobile                     0
username                       0
num_cities                     0
num_reviews                    0
num_type_reviews               0
author_num_helpful_votes       0
author_id                      0
service                        0
cleanliness                    0
overall                        0
value                          0
location                      16
sleep_quality                 16
rooms                          0
hotel_class                    0
region_id                      0
url                            0
address                        0
type                           0
id_y                           0
name                           0
region                         0
street-address                 1
postal-code                 1681
locality  

In [63]:
# fill the postal_code
region_list = list(df.region.value_counts().index)

In [78]:
for i in region_list:
    df.loc[(df['postal-code'].isnull())&(df.region == i),'postal-code'] = list(df[df.region == i]['postal-code'].value_counts(ascending = False).index)[0]

/Users/siyuan/opt/anaconda3/lib/python3.8/site-packages/pandas/core/indexing.py:966: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


In [80]:
# street address is not using, so we can ignore it. So do the location and the sleep quality
df.isna().sum()

title                        0
text                         0
offering_id                  0
num_helpful_votes            0
date                         0
id_x                         0
via_mobile                   0
username                     0
num_cities                   0
num_reviews                  0
num_type_reviews             0
author_num_helpful_votes     0
author_id                    0
service                      0
cleanliness                  0
overall                      0
value                        0
location                    16
sleep_quality               16
rooms                        0
hotel_class                  0
region_id                    0
url                          0
address                      0
type                         0
id_y                         0
name                         0
region                       0
street-address               1
postal-code                  0
locality                     0
dtype: int64

In [87]:
# we see those NaN comes from 0 overall rating - we gonna drop it 
df.loc[df['location'].isnull()][['location','overall']]

,location,overall
63605,NaN,0.0
71654,NaN,0.0
85710,NaN,0.0
89118,NaN,0.0
101215,NaN,0.0
119012,NaN,0.0
193799,NaN,0.0
273409,NaN,0.0
347982,NaN,0.0
348930,NaN,0.0


In [89]:
df = df[df.overall != 0]

In [92]:
df.isna().sum()

title                       0
text                        0
offering_id                 0
num_helpful_votes           0
date                        0
id_x                        0
via_mobile                  0
username                    0
num_cities                  0
num_reviews                 0
num_type_reviews            0
author_num_helpful_votes    0
author_id                   0
service                     0
cleanliness                 0
overall                     0
value                       0
location                    0
sleep_quality               0
rooms                       0
hotel_class                 0
region_id                   0
url                         0
address                     0
type                        0
id_y                        0
name                        0
region                      0
street-address              1
postal-code                 0
locality                    0
dtype: int64

In [93]:
df.to_csv('review_offer_combined.csv')